## Outil de concaténation et de classification par cohortes des courbes de charge
---

In [2]:
import pandas as pd
import os

### Importation des données

Importation des pods de chaque cohorte.


In [2]:
CheminC=input("Chemin de l'excel contenant les numéros des pods de chaque cohorte : ")
Chemin_Cohortes=r"{}".format(CheminC)
Noms_Cohortes=["LE CEDRE","CHANEL","Toulouse lot 5","AIH","St-Etienne","Ze Plug AO"]
Cohortes=pd.read_excel(Chemin_Cohortes,sheet_name='AO',usecols=Noms_Cohortes)

Importation des chemins des fichiers csv d'ENEDIS. Il faut renseigner le chemin du dossier avec toutes les courbes de charge.  

In [6]:
CheminD=input("Chemin du dossier contenant les csv des courbes de charge : ")
Chemin_dossier_CDC=r"{}".format(CheminD)
chemins = []
nb=0
for racine, dirs, fichiers in os.walk(f"{Chemin_dossier_CDC}"):
    for file in fichiers:
        nb+=1
#La condition if permet de ne choisir que les fichiers csv qui commencent par Enedis et dont la taille est supérieure à 19ko
        if file.endswith(".csv") and file.startswith("Enedis") and os.path.getsize(racine+"\\"+file)>=19000:
            s = os.path.join(racine, file)
            chemins.append(s)
print(f"Nombre de fichiers dans le dossier = {nb}\nNombre de fichiers csv d'ENEDIS pris = {len(chemins)}")

Nombre de fichiers dans le dossier = 2983
Nombre de fichiers csv d'ENEDIS pris = 2962


### Classification des CdC selon leur cohorte et supression des doublons

In [4]:
def listes_chemins_cohortes(chemins,Cohortes):
    l1,l2,l3,l4,l5,l6,lnone,l,luniques,ldoublons=[],[],[],[],[],[],[],[],[],[]
    seen=set()

# création d'une liste de tuples formés des chemins csv des PODs et de leur numéro
    for chemin_csv in chemins:
        num_pod=pd.read_csv(chemin_csv,sep=';',nrows=1,usecols=[0]).loc[0,"Identifiant PRM"]
        l.append((chemin_csv,int(num_pod)))

#création de deux listes : luniques est la liste des tuples sans doublons 
#et ldoublons est la liste des tuples qui étaient doublons dans la liste l et qui ne sont pas dans luniques
    for tup in l:
        if tup[1] not in seen:
            seen.add(tup[1])
            luniques.append(tup)
        else:
            ldoublons.append(tup)

#Classification des tuples uniques par cohorte            
    for ch in luniques:
        if ch[1] in Cohortes['LE CEDRE'].tolist():
            l1.append((ch[0],ch[1]))
        elif ch[1] in Cohortes['CHANEL'].tolist():
            l2.append((ch[0],ch[1]))
        elif ch[1] in Cohortes['Toulouse lot 5'].tolist():
            l3.append((ch[0],ch[1]))
        elif ch[1] in Cohortes['AIH'].tolist():
            l4.append((ch[0],ch[1]))
        elif ch[1] in Cohortes['St-Etienne'].tolist():
            l5.append((ch[0],ch[1]))
        elif ch[1] in Cohortes['Ze Plug AO'].tolist():
            l6.append((ch[0],ch[1]))
        else :
            lnone.append((ch[0],ch[1]))

    return ldoublons,l1,l2,l3,l4,l5,l6,lnone

In [5]:
%%time
liste_doublons,l_LECEDRE,l_CHANEL,l_Tlse,l_AIH,l_StEtienne,l_Ze,lnone=listes_chemins_cohortes(chemins,Cohortes)

Wall time: 15.3 s


### Concaténation, rééchantillonnage temporel et sommation pour chaque cohorte

In [6]:
def concaténationPODS(lchemins):
    #Création d'un dataframe vide dfc avec un dateTimeindex allant de nov 2020 à fev 2023 avec un pas de 30 minutes
    dfc=pd.DataFrame(index=pd.date_range(start=pd.Timestamp('2020-11-11 00:00:00', tz='Europe/Paris'),
                                         end=pd.Timestamp('2023-02-17 00:00:00', tz='Europe/Paris'),freq='30T'))
    
    # Importation des données de chaque POD de la cohorte et concaténation dans le dataframe dfc
    for path,num in lchemins:
        df = pd.read_csv(path, sep=';', skiprows=2, usecols=['Horodate', 'Valeur']) 
        df = df.set_index('Horodate') 
        df.index = pd.to_datetime(df.index,utc=True).tz_convert('Europe/Paris')
        df = df.resample('30T').mean() # rééchantillonnage au pas demi-horaire de l'horizon temporel
        dfc = pd.concat([dfc, df.rename(columns={'Valeur': num})], axis=1) #Ajout au dataframe vide la colonne des valeur du POD

    #renvoi de la somme de toutes les valeurs des PODs de la cohorte pour chaque pas demi-horaire
    return dfc.sum(axis=1) 

In [8]:
%%time
dfLECEDRE= concaténationPODS(l_LECEDRE)
dfAIH=concaténationPODS(l_AIH)
dfCHANEL=concaténationPODS(l_CHANEL)
dfStEtienne=concaténationPODS(l_StEtienne)
dfTlse=concaténationPODS(l_Tlse)
dfZe=concaténationPODS(l_Ze)

Wall time: 7min 1s


In [9]:
%%time
dfnone=concaténationPODS(lnone)

Wall time: 19min 57s


### Concaténation et exportations des données

Création d'un dataframe contenant les valeurs de consommation au pas demi horaire classées par cohorte

In [10]:
ldataframes=[dfLECEDRE,dfCHANEL,dfTlse,dfAIH,dfStEtienne,dfZe,dfnone]
dftot=pd.concat(ldataframes,axis=1,verify_integrity=True) #agrégation des dataframes
dftot=dftot.rename(columns={i:Noms_Cohortes[i] for i in range(len(Noms_Cohortes))})
dftot=dftot.rename(columns={6:"Autre"})
dftot.index=dftot.index.tz_localize(None) #conversion des dates au format tz-naive car excel ne prend pas en charge le format tz-aware

Création d'un dataframe contenant les numéros et les chemins des PODs doublons supprimés

In [11]:
dfdoublons=pd.DataFrame({"N° PODS en doubles supprimés":[i[1] for i in liste_doublons],"chemins":[i[0] for i in liste_doublons]})

Création d'un excel avec une feuille contenant dftot et une feuille contenant dfdoublons

In [12]:
%%time
with pd.ExcelWriter(Chemin_dossier_CDC+"\\CdC PODs jaunes.xlsx") as writer:
    dftot.to_excel(writer,sheet_name="CdC", startrow=1)
    dfdoublons.to_excel(writer,sheet_name="Doublons")

Wall time: 5.13 s
